In [68]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import sys 
sys.path.append('../')
import src.features.feature_cleaning as feature_cleaning

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


#sklearn models
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
#sklearn other
import graphviz 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, log_loss
from mlxtend.plotting import plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [61]:
df, fieldofdegree_df, SOCP_labels, schl_labels, major_majors, NAICSP_labels_df, MAJ_NAICSP_labels_df = feature_cleaning.load_dfs()

In [62]:
youngemp_df = feature_cleaning.clean_that_target(df, SOCP_labels)
youngemp_df = feature_cleaning.single_occ_target(youngemp_df)
edu_df = feature_cleaning.create_edu_df(youngemp_df, fieldofdegree_df, schl_labels, major_majors)

../src/features/feature_cleaning.py:81: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  SOCPdf = df.dropna(axis='index', subset=['SOCP'])[df.SOCP != '999920']


Number of employed people: 218454
Percent employed people: 0.5785711448056677
Number of young employed people: 77406
Percent young employed people(out of all PUMS): 0.20500827650135733
Number of emp cats: 23
Number of degree fields present (max 173): 173


/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
../src/features/feature_cleaning.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  edu_df['SCHL_ord'] = edu_df.SCHL.astype(int)
../src/features/feature_cleaning.py:152: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

before dummies:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 77406 entries, 0 to 77405
Data columns (total 14 columns):
SERIALNO            77406 non-null int64
SOCP                77406 non-null object
MAJ_SOCP            77406 non-null object
MAJ_SOCP_labels     77406 non-null object
MAJ_SOCP_15         77406 non-null int64
FOD1P               77406 non-null object
FOD2P               77406 non-null object
FOD1P_labels        77406 non-null object
FOD2P_labels        77406 non-null object
SCHL                77406 non-null object
SCHL_labels         77406 non-null object
SCHL_ord            77406 non-null int64
FOD1P_MAJ           77406 non-null int64
FOD1P_MAJ_labels    77406 non-null object
dtypes: int64(4), object(10)
memory usage: 53.1 MB
None


In [59]:
edu_df.sample(3)

,SERIALNO,SOCP,MAJ_SOCP,MAJ_SOCP_labels,MAJ_SOCP_15,FOD1P,FOD2P,FOD1P_labels,FOD2P_labels,SCHL,SCHL_labels,SCHL_ord,FOD1P_MAJ,FOD1P_MAJ_labels,"FOD1P_MAJ__Agriculture, agriculture operations, and related sciences",FOD1P_MAJ__Architecture and related services,"FOD1P_MAJ__Area, ethnic, cultural, gender, and group studies",FOD1P_MAJ__Biological and biomedical sciences,"FOD1P_MAJ__Business, management, marketing, and related support services","FOD1P_MAJ__Communication, journalism, and related programs",FOD1P_MAJ__Communications technologies/technicians and support services,FOD1P_MAJ__Computer and information sciences and support services,FOD1P_MAJ__Construction trades,FOD1P_MAJ__Education,FOD1P_MAJ__Engineering,FOD1P_MAJ__Engineering technologies and engineering-related fields,FOD1P_MAJ__English language and literature/letters,FOD1P_MAJ__Family and consumer sciences/human sciences,"FOD1P_MAJ__Foreign languages, literatures, and linguistics",FOD1P_MAJ__Health professions and related programs,FOD1P_MAJ__History,"FOD1P_MAJ__Homeland security, law enforcement, firefighting and related protective services",FOD1P_MAJ__Legal professions and studies,"FOD1P_MAJ__Liberal arts and sciences, general studies and humanities",FOD1P_MAJ__Library science,FOD1P_MAJ__Mathematics and statistics,FOD1P_MAJ__Mechanic and repair technologies/technicians,"FOD1P_MAJ__Military science, leadership and operational art",FOD1P_MAJ__Multi/interdisciplinary studies,FOD1P_MAJ__Natural resources and conservation,FOD1P_MAJ__No major,"FOD1P_MAJ__Parks, recreation, leisure, and fitness studies",FOD1P_MAJ__Personal and culinary services,FOD1P_MAJ__Philosophy and religious studies,FOD1P_MAJ__Physical sciences,FOD1P_MAJ__Psychology,FOD1P_MAJ__Public administration and social service professions,FOD1P_MAJ__Science technologies/technicians,FOD1P_MAJ__Social sciences,FOD1P_MAJ__Theology and religious vocations,FOD1P_MAJ__Transportation and materials moving,FOD1P_MAJ__Visual and performing arts,FOD1P__Accounting,FOD1P__Actuarial Science,FOD1P__Advertising And Public Relations,FOD1P__Aerospace Engineering,FOD1P__Agricultural Economics,FOD1P__Agriculture Production And Management,FOD1P__Animal Sciences,FOD1P__Anthropology And Archeology,FOD1P__Applied Mathematics,FOD1P__Architectural Engineering,FOD1P__Architecture,FOD1P__Area Ethnic And Civilization Studies,FOD1P__Art And Music Education,FOD1P__Art History And Criticism,FOD1P__Astronomy And Astrophysics,FOD1P__Atmospheric Sciences And Meteorology,FOD1P__Biochemical Sciences,FOD1P__Biological Engineering,FOD1P__Biology,FOD1P__Biomedical Engineering,FOD1P__Botany,FOD1P__Business Economics,FOD1P__Business Management And Administration,FOD1P__Chemical Engineering,FOD1P__Chemistry,FOD1P__Civil Engineering,FOD1P__Clinical Psychology,FOD1P__Cognitive Science And Biopsychology,FOD1P__Commercial Art And Graphic Design,FOD1P__Communication Disorders Sciences And Services,FOD1P__Communication Technologies,FOD1P__Communications,FOD1P__Community And Public Health,FOD1P__Composition And Rhetoric,FOD1P__Computer Administration Management And Security,FOD1P__Computer And Information Systems,FOD1P__Computer Engineering,FOD1P__Computer Networking And Telecommunications,FOD1P__Computer Programming And Data Processing,FOD1P__Computer Science,FOD1P__Construction Services,FOD1P__Cosmetology Services And Culinary Arts,FOD1P__Counseling Psychology,FOD1P__Court Reporting,FOD1P__Criminal Justice And Fire Protection,FOD1P__Criminology,FOD1P__Drama And Theater Arts,FOD1P__Early Childhood Education,FOD1P__Ecology,FOD1P__Economics,FOD1P__Educational Administration And Supervision,FOD1P__Educational Psychology,FOD1P__Electrical Engineering,FOD1P__Electrical Engineering Technology,"FOD1P__Electrical, Mechanical, And Precision Technologies And Production",FOD1P__Elementary Education,FOD1P__Engineering And Industrial Management,FOD1P__Engineering Mechanics Physics And Science,FOD1P__Engineering Technologies,FOD1P__English Language And Literature,FOD1P__Enviro

In [63]:
# split the data
X = edu_df.drop(columns=[ 'SERIALNO', 'FOD1P', 'FOD2P','SOCP','MAJ_SOCP','MAJ_SOCP_labels', 
                'MAJ_SOCP_15','FOD1P_labels','FOD2P_labels','SCHL',
                'SCHL_labels','FOD1P_MAJ_labels', 'FOD1P_MAJ'])
y = edu_df.loc[:,'MAJ_SOCP_15']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3, 
                                                    random_state=42)

In [73]:
X_train.shape

(54184, 388)

In [64]:
X_train.sample(5)

,SCHL_ord,"FOD1P_MAJ__Agriculture, agriculture operations, and related sciences",FOD1P_MAJ__Architecture and related services,"FOD1P_MAJ__Area, ethnic, cultural, gender, and group studies",FOD1P_MAJ__Biological and biomedical sciences,"FOD1P_MAJ__Business, management, marketing, and related support services","FOD1P_MAJ__Communication, journalism, and related programs",FOD1P_MAJ__Communications technologies/technicians and support services,FOD1P_MAJ__Computer and information sciences and support services,FOD1P_MAJ__Construction trades,FOD1P_MAJ__Education,FOD1P_MAJ__Engineering,FOD1P_MAJ__Engineering technologies and engineering-related fields,FOD1P_MAJ__English language and literature/letters,FOD1P_MAJ__Family and consumer sciences/human sciences,"FOD1P_MAJ__Foreign languages, literatures, and linguistics",FOD1P_MAJ__Health professions and related programs,FOD1P_MAJ__History,"FOD1P_MAJ__Homeland security, law enforcement, firefighting and related protective services",FOD1P_MAJ__Legal professions and studies,"FOD1P_MAJ__Liberal arts and sciences, general studies and humanities",FOD1P_MAJ__Library science,FOD1P_MAJ__Mathematics and statistics,FOD1P_MAJ__Mechanic and repair technologies/technicians,"FOD1P_MAJ__Military science, leadership and operational art",FOD1P_MAJ__Multi/interdisciplinary studies,FOD1P_MAJ__Natural resources and conservation,FOD1P_MAJ__No major,"FOD1P_MAJ__Parks, recreation, leisure, and fitness studies",FOD1P_MAJ__Personal and culinary services,FOD1P_MAJ__Philosophy and religious studies,FOD1P_MAJ__Physical sciences,FOD1P_MAJ__Psychology,FOD1P_MAJ__Public administration and social service professions,FOD1P_MAJ__Science technologies/technicians,FOD1P_MAJ__Social sciences,FOD1P_MAJ__Theology and religious vocations,FOD1P_MAJ__Transportation and materials moving,FOD1P_MAJ__Visual and performing arts,FOD1P__Accounting,FOD1P__Actuarial Science,FOD1P__Advertising And Public Relations,FOD1P__Aerospace Engineering,FOD1P__Agricultural Economics,FOD1P__Agriculture Production And Management,FOD1P__Animal Sciences,FOD1P__Anthropology And Archeology,FOD1P__Applied Mathematics,FOD1P__Architectural Engineering,FOD1P__Architecture,FOD1P__Area Ethnic And Civilization Studies,FOD1P__Art And Music Education,FOD1P__Art History And Criticism,FOD1P__Astronomy And Astrophysics,FOD1P__Atmospheric Sciences And Meteorology,FOD1P__Biochemical Sciences,FOD1P__Biological Engineering,FOD1P__Biology,FOD1P__Biomedical Engineering,FOD1P__Botany,FOD1P__Business Economics,FOD1P__Business Management And Administration,FOD1P__Chemical Engineering,FOD1P__Chemistry,FOD1P__Civil Engineering,FOD1P__Clinical Psychology,FOD1P__Cognitive Science And Biopsychology,FOD1P__Commercial Art And Graphic Design,FOD1P__Communication Disorders Sciences And Services,FOD1P__Communication Technologies,FOD1P__Communications,FOD1P__Community And Public Health,FOD1P__Composition And Rhetoric,FOD1P__Computer Administration Management And Security,FOD1P__Computer And Information Systems,FOD1P__Computer Engineering,FOD1P__Computer Networking And Telecommunications,FOD1P__Computer Programming And Data Processing,FOD1P__Computer Science,FOD1P__Construction Services,FOD1P__Cosmetology Services And Culinary Arts,FOD1P__Counseling Psychology,FOD1P__Court Reporting,FOD1P__Criminal Justice And Fire Protection,FOD1P__Criminology,FOD1P__Drama And Theater Arts,FOD1P__Early Childhood Education,FOD1P__Ecology,FOD1P__Economics,FOD1P__Educational Administration And Supervision,FOD1P__Educational Psychology,FOD1P__Electrical Engineering,FOD1P__Electrical Engineering Technology,"FOD1P__Electrical, Mechanical, And Precision Technologies And Production",FOD1P__Elementary Education,FOD1P__Engineering And Industrial Management,FOD1P__Engineering Mechanics Physics And Science,FOD1P__Engineering Technologies,FOD1P__English Language And Literature,FOD1P__Environmental Engineering,FOD1P__Environmental Science,FOD1P__Family And Consumer Sciences,FOD1P__Film Video And Photographic Arts,FOD1P__F

In [80]:
y_train.shape

(54184,)

In [107]:

# model pipelines
#-----------------------------------
#-------------linear
pipe_lr = Pipeline([('scl', StandardScaler()),
			('clf', LogisticRegression(random_state=42))])

pipe_lr_l2 = Pipeline([('scl', StandardScaler()),
			('clf', LogisticRegression(random_state=42))])

pipe_sgd = Pipeline([('scl', StandardScaler()),
			('clf', SGDClassifier(random_state=42))])


#-------------trees
pipe_dt = Pipeline([('clf', DecisionTreeClassifier(random_state=42))])

pipe_rf = Pipeline([('clf', RandomForestClassifier(random_state=42))])

pipe_rf_scl = Pipeline([('scl', StandardScaler()),
			('clf', RandomForestClassifier(random_state=42))])

pipe_gb = Pipeline([('clf', GradientBoostingClassifier(random_state=42))])


#-------------SVM
pipe_svm = Pipeline([('scl', StandardScaler()),
			('clf', SVC(random_state=42))])


#-------------KNN
pipe_knn = Pipeline([('clf', KNeighborsClassifier())])

pipe_knn_scl = Pipeline([('scl', StandardScaler()),
			('clf', KNeighborsClassifier())])

#-----------------------------------

In [108]:

# grid search params
param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range_fl = [1.0, 0.5, 0.1]
max_depth = [10,100,1000,10000]
alpha_range = [.1, .001, .00001, .000001]
gamma_range = [.1, 1, 10]

#-------------linear
grid_params_lr = [{'clf__penalty': ['l1'],
		'clf__C': param_range_fl,
		'clf__solver': ['liblinear', ],  #,'saga'
        #'clf__multi_class': ['ovr', 'multinomial', 'auto'],
        'clf__class_weight': [None, 'balanced']}] 

grid_params_lr_l2 = [{'clf__penalty': ['l2'],
		'clf__C': param_range_fl,
		'clf__solver': ['newton-cg', 'lbfgs', 'liblinear'],  #, 'sag'
        #'clf__multi_class': ['ovr', 'multinomial', 'auto'],
        'clf__class_weight': [None, 'balanced']}]

grid_params_sgd = [{'clf__loss': ['hinge', 'log', 'perceptron'],
		'clf__alpha': alpha_range,
		'clf__penalty': ['l1', 'l2', 'elasticnet'],
        'clf__class_weight': [None, 'balanced']}] 

#-------------trees
grid_params_dt = [{'clf__criterion': ['gini', 'entropy'],
		'clf__min_samples_leaf': param_range,
		'clf__max_depth': max_depth,
		'clf__min_samples_split': param_range[1:],
        'clf__class_weight': [None, 'balanced']}]

grid_params_rf = [{'clf__criterion': ['gini', 'entropy'],
		'clf__min_samples_leaf': param_range,
		'clf__max_depth': max_depth,
		'clf__min_samples_split': param_range[1:],
        'clf__class_weight': [None, 'balanced', 'balanced_subsample']}]

grid_params_gb = [{'clf__loss': ['deviance', 'exponential'],
		'clf__learning_rate': alpha_range,
		'clf__n_estimators': max_depth,
		'clf__subsample': param_range_fl}]

#-------------SVM
grid_params_svm = [{'clf__kernel': ['linear', 'rbf', 'poly'],
        'clf__degree': param_range[1:],
        'clf__gamma': gamma_range,
        'clf__C': gamma_range,
        'clf__class_weight': [None, 'balanced']}]

#-------------KNN
grid_params_knn = [{'clf__n_neighbors': param_range}]

In [109]:
# Construct grid searches
jobs = -1
verbose = 10

#-------------linear
gs_lr = GridSearchCV(estimator=pipe_lr,
			param_grid=grid_params_lr,
			scoring='f1_micro',
			cv=10,
            n_jobs=jobs,
            verbose=verbose) 

gs_lr_l2 = GridSearchCV(estimator=pipe_lr_l2,
			param_grid=grid_params_lr_l2,
			scoring='f1_micro',
			cv=10,
            n_jobs=jobs,
            verbose=verbose)

gs_sgd = GridSearchCV(estimator=pipe_sgd,
			param_grid=grid_params_sgd,
			scoring='f1_micro',
			cv=10,
            verbose=verbose)

    
#-------------trees    
gs_dt = GridSearchCV(estimator=pipe_rf,
			param_grid=grid_params_dt,
			scoring='f1_micro',
			cv=10, 
			n_jobs=jobs,
            verbose=verbose)

gs_rf = GridSearchCV(estimator=pipe_rf,
			param_grid=grid_params_rf,
			scoring='f1_micro',
			cv=10, 
			n_jobs=jobs,
            verbose=verbose)

gs_rf_scl = GridSearchCV(estimator=pipe_rf_scl,
			param_grid=grid_params_rf,
			scoring='f1_micro',
			cv=10, 
			n_jobs=jobs,
            verbose=verbose)

gs_gb = GridSearchCV(estimator=pipe_gb,
			param_grid=grid_params_gb,
			scoring='f1_micro',
			cv=10, 
            verbose=verbose)

#-------------SVM

gs_svm = GridSearchCV(estimator=pipe_svm,
			param_grid=grid_params_svm,
			scoring='f1_micro',
			cv=10,
			n_jobs=jobs,
            verbose=verbose)

#-------------KNN
gs_knn = GridSearchCV(estimator=pipe_knn,
			param_grid=grid_params_knn,
			scoring='f1_micro',
			cv=10,
			n_jobs=jobs,
            verbose=verbose)

gs_knn_scl = GridSearchCV(estimator=pipe_knn_scl,
			param_grid=grid_params_knn,
			scoring='f1_micro',
			cv=10,
			n_jobs=jobs,
            verbose=verbose)


In [ ]:
# List of pipelines for ease of iteration
grids = [gs_lr, gs_lr_l2, gs_sgd, 
         gs_dt, gs_rf, gs_rf_scl, gs_gb, 
         gs_svm, gs_knn, gs_knn_scl]

# Dictionary of pipelines and classifier types for ease of reference
grid_dict = {0: 'Logistic Regression w/ L1', 1: 'LogisticRegression w/ L2', 2: 'SGDClassifier', 3: 'DecisionTreeClassifier',
             4: 'Random Forest', 5: 'Random Forest w/ Scaling', 6: 'GradientBoostingClassifier',
            7:'SVC', 8: 'KNeighborsClassifier', 9:'KNeighborsClassifier w/ Scaling'}

# Fit the grid search objects
print('Performing model optimizations...')
best_f1_micro = 0.0
best_clf = 0
best_gs = ''
for idx, gs in enumerate(grids):
	print('\nEstimator: %s' % grid_dict[idx])
	# Fit grid search
	gs.fit(X_train, y_train)
    
	# Best params
	print('Best params: %s' % gs.best_params_)
    
	# Best training data f1
	print('Best training f1: %.3f' % gs.best_score_)
    
	# Predict on test data with best params
	y_pred = gs.predict(X_test)
    
	# Test data accuracy of model with best params
	print('Test set f1 score for best params: %.3f ' % f1_score(y_test, y_pred))
    
	# Track best (highest test f1) model
	if f1_score(y_test, y_pred) > best_f1_micro:
		best_f1_micro = f1_score(y_test, y_pred)
		best_gs = gs
		best_clf = idx
print('\nClassifier with best test set f1: %s' % grid_dict[best_clf])

# Save best grid search pipeline to file
dump_file = 'best_model_no_feat_sel_extr_occ_15.pkl'
joblib.dump(best_gs, dump_file, compress=1)
print('\nSaved %s grid search pipeline to file: %s' % (grid_dict[best_clf], dump_file))

Performing model optimizations...

Estimator: Logistic Regression w/ L1
Fitting 10 folds for each of 6 candidates, totalling 60 fits
[CV] clf__C=1.0, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV] clf__C=1.0, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV] clf__C=1.0, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV] clf__C=1.0, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV] clf__C=1.0, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV] clf__C=1.0, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV] clf__C=1.0, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV] clf__C=1.0, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV] clf__C=1.0, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV] clf__C=1.0, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV] clf__C=1.0, clf__class_weight=balanced, clf__penalty

[CV] clf__C=0.1, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV]  clf__C=0.5, clf__class_weight=balanced, clf__penalty=l1, clf__solver=liblinear, score=0.862705296180107, total=30.1min
[CV] clf__C=0.1, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV]  clf__C=0.1, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear, score=0.9671526111828751, total= 5.8min
[CV] clf__C=0.1, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV]  clf__C=0.1, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear, score=0.9682598265362613, total= 4.3min
[CV] clf__C=0.1, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear 
[CV]  clf__C=0.1, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear, score=0.9673311184939092, total= 4.2min
[CV] clf__C=0.1, clf__class_weight=balanced, clf__penalty=l1, clf__solver=liblinear 
[CV]  clf__C=0.1, clf__class_weight=None, clf__penalty=l1, clf__solver=liblinear, score=0.9688076781

[Parallel(n_jobs=-1)]: Done  50 out of  60 | elapsed: 125.0min remaining: 25.0min


[CV]  clf__C=0.1, clf__class_weight=balanced, clf__penalty=l1, clf__solver=liblinear, score=0.8556929322753275, total=12.0min
[CV]  clf__C=0.5, clf__class_weight=balanced, clf__penalty=l1, clf__solver=liblinear, score=0.8593836501199483, total=43.6min
[CV]  clf__C=0.1, clf__class_weight=balanced, clf__penalty=l1, clf__solver=liblinear, score=0.8567737172388336, total= 6.7min
[CV]  clf__C=0.1, clf__class_weight=balanced, clf__penalty=l1, clf__solver=liblinear, score=0.8553238604908655, total= 8.1min
[CV]  clf__C=0.1, clf__class_weight=balanced, clf__penalty=l1, clf__solver=liblinear, score=0.8654733345635726, total= 9.6min
[CV]  clf__C=0.1, clf__class_weight=balanced, clf__penalty=l1, clf__solver=liblinear, score=0.8693244739756367, total= 6.7min
[CV]  clf__C=0.1, clf__class_weight=balanced, clf__penalty=l1, clf__solver=liblinear, score=0.8488651042627791, total= 9.2min
[CV]  clf__C=0.1, clf__class_weight=balanced, clf__penalty=l1, clf__solver=liblinear, score=0.8617315857485692, total=

[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 130.0min finished


...

[CV]  clf__alpha=0.001, clf__class_weight=None, clf__loss=hinge, clf__penalty=elasticnet, score=0.9675216829673371, total=   1.3s
[CV] clf__alpha=0.001, clf__class_weight=None, clf__loss=hinge, clf__penalty=elasticnet 


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[CV]  clf__alpha=0.001, clf__class_weight=None, clf__loss=hinge, clf__penalty=elasticnet, score=0.9642000369071785, total=   1.4s
[CV] clf__alpha=0.001, clf__class_weight=None, clf__loss=hinge, clf__penalty=elasticnet 


/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


[CV]  clf__alpha=0.001, clf__class_weight=None, clf__loss=hinge, clf__penalty=elasticnet, score=0.9643845727994095, total=   1.4s
[CV] clf__alpha=0.001, clf__class_weight=None, clf__loss=hinge, clf__penalty=elasticnet 


In [ ]:
PCA


from sklearn.decomposition import PCA

pipe_lr_pca = Pipeline([('scl', StandardScaler()),
			('pca', PCA(n_components=2)),
			('clf', LogisticRegression(random_state=42))])

pipe_sgd_pca = Pipeline([('scl', StandardScaler()),
			('pca', PCA(n_components=2)),
			('clf', SGDClassifier(random_state=42))])

pipe_rf_pca = Pipeline([('scl', StandardScaler()),
			('pca', PCA(n_components=2)),
			('clf', RandomForestClassifier(random_state=42))])


pipe_svm_pca = Pipeline([('scl', StandardScaler()),
			('pca', PCA(n_components=2)),
			('clf', SVC(random_state=42))])

------------------

gs_lr_pca = GridSearchCV(estimator=pipe_lr_pca,
			param_grid=grid_params_lr,
			scoring='f1_micro',
			cv=10,
            verbose=verbose)

gs_sgd_pca = GridSearchCV(estimator=pipe_sgd_pca,
			param_grid=grid_params_sgd,
			scoring='f1_micro',
			cv=10,
            verbose=verbose)

gs_rf_pca = GridSearchCV(estimator=pipe_rf_pca,
			param_grid=grid_params_rf,
			scoring='f1_micro',
			cv=10, 
			n_jobs=jobs,
            verbose=verbose)

gs_svm_pca = GridSearchCV(estimator=pipe_svm_pca,
			param_grid=grid_params_svm,
			scoring='f1_micro',
			cv=10,
			n_jobs=jobs,
            verbose=verbose)






In [ ]:
conda upgrade matplotlib